In [ ]:
from os import path
import cognee
import dspy
from dspy.datasets import HotPotQA

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

await cognee.prune.prune_system()

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url = "http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(rm = colbertv2_wiki17_abstracts)

dataset = HotPotQA(
    train_seed = 1,
    train_size = 10,
    eval_seed = 2023,
    dev_size = 0,
    test_size = 0,
    keep_details = True,
)

texts_to_add = []

for train_case in dataset.train:
    train_case_text = "\r\n".join(" ".join(context_sentences) for context_sentences in train_case.get("context")["sentences"])

    texts_to_add.append(train_case_text)

dataset_name = "train_dataset"
await cognee.add(texts_to_add, dataset_name)


In [ ]:
from os import path
import cognee

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

print(cognee.datasets.list_datasets())

train_dataset = cognee.datasets.query_data('train_dataset')
print(len(train_dataset))

In [ ]:
from os import path
import cognee

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

await cognee.cognify('train_dataset')

In [ ]:
from networkx.algorithms import dfs_successors
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType

graph_client = get_graph_client(GraphDBType.NETWORKX, "7e5137a5-2835-464d-94e8-edfc872b5944.pkl")
await graph_client.load_graph_from_file()
graph = graph_client.graph

successors_map = dfs_successors(graph, source = "Beorhtnoð_Kent")

for parent_id, child_ids in successors_map.items():
    parent_label = graph.nodes[parent_id]["entity_name"]

    for child_id in child_ids:
        relationship_label = graph[parent_id][child_id].get(0)["relationship_name"]
        child_label = graph.nodes[child_id]["entity_name"]

        print(parent_label + " " + relationship_label + " " + child_label)

In [ ]:
import dsp
import dspy
from dspy.datasets import HotPotQA
from dspy.evaluate.evaluate import Evaluate
from cognee.modules.search.CogneeSearch import CogneeSearch

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url = "http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(rm = colbertv2_wiki17_abstracts)

dataset = HotPotQA(
    train_seed = 1,
    train_size = 10,
    eval_seed = 2023,
    dev_size = 0,
    test_size = 0,
    keep_details = True,
)

eval_set = [x.with_inputs('question') for x in dataset.train[:10]]

def evaluate_answer(example, prediction):
    return dsp.answer_match(example.answer, [prediction.answer], frac = 0.8) or \
        dsp.passage_match([example.answer], [prediction.answer])

evaluate = Evaluate(devset = eval_set, metric = evaluate_answer, num_threads = 1, display_progress = True)

program = CogneeSearch()

baseline_eval_score = evaluate(program, devset = eval_set)
print("Baseline eval score: %s", baseline_eval_score)


In [ ]:
graph = await cognee.cognify(dataset_name)

from cognee.utils import render_graph

await render_graph(graph)

In [ ]:
from uuid import uuid4
from cognee.shared.data_models import KnowledgeGraph, Node, Edge
from cognee.modules.cognify.dspy import are_all_nodes_connected, are_all_nodes_and_edges_valid

nodes = [
    Node(id = str(uuid4()), entity_name = "Jade", entity_type = "Person"),
    Node(id = str(uuid4()), entity_name = "Nevada", entity_type = "Location"),
    Node(id = str(uuid4()), entity_name = "Madagascar", entity_type = "Location"),
    Node(id = str(uuid4()), entity_name = "Berlin", entity_type = "Location"),
    Node(id = str(uuid4()), entity_name = "Turtles", entity_type = "Time Period"),
    Node(id = str(uuid4()), entity_name = "6 months", entity_type = "Animal"),
    Node(id = str(uuid4()), entity_name = "Berghain", entity_type = "Venue"),
]

edges = [
    Edge(source_node_id = nodes[0].id, target_node_id = nodes[1].id, relationship_name = "lives in"),
    Edge(source_node_id = nodes[0].id, target_node_id = nodes[2].id, relationship_name = "traveled to"),
    Edge(source_node_id = nodes[0].id, target_node_id = nodes[3].id, relationship_name = "lived in"),
    Edge(source_node_id = nodes[4].id, target_node_id = nodes[2].id, relationship_name = "live on"),
    Edge(source_node_id = nodes[5].id, target_node_id = nodes[3].id, relationship_name = "in a time period of"),
    Edge(source_node_id = nodes[6].id, target_node_id = nodes[4].id, relationship_name = "is located in"),
]

graph = KnowledgeGraph(nodes = nodes, edges = edges)

print(are_all_nodes_connected(graph))
print(are_all_nodes_and_edges_valid(graph))


In [ ]:
import dspy
from dspy.datasets import HotPotQA
from dspy.evaluate.evaluate import Evaluate
from cognee.modules.cognify.train import train

generate_graph = train()

def evaluate_answer(example, prediction, frac = 0.8):
    return dsp.answer_match(example.answer, [prediction.answer], frac = frac) or \
        dsp.passage_match([example.answer], [prediction.answer])

hotpot_dataset = HotPotQA(train_seed = 1, eval_seed = 2023, dev_size = 20)
devset = [x.with_inputs('question') for x in hotpot_dataset.dev][:20]

evaluate_on_hotpotqa = Evaluate(devset = devset, metric = evaluate_answer, num_threads = 10, display_progress = True, display_table = 5)

score = evaluate_on_hotpotqa(generate_graph, devset = devset)

print(score)


In [ ]:
import networkx as nx
import pandas as pd
import graphistry
from cognee.config import Config
from cognee.utils import render_graph
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType

config = Config()
config.load()

graphistry.register(
    api = 3,
    username = config.graphistry_username,
    password = config.graphistry_password
)

graph_client = await get_graph_client(GraphDBType.NETWORKX, "cognee_graph.pkl")
graph = graph_client.graph

edges = nx.to_pandas_edgelist(graph)

nodes_data = [{
    "id": node_id,
    "entity_name": node["label"] if "label" in node else (node["name"] if "name" in node else node_id),
} for (node_id, node) in graph.nodes(data = True)]

nodes = pd.DataFrame(nodes_data)

plotter = graphistry.edges(edges, source = "source", destination = "target").nodes(nodes, "id")

plotter.bind(edge_title = "relationship_name", edge_label = "relationship_name", point_title = "label", point_label = "label")
url = plotter.plot(render = False, as_files = True)
print(f"Graph is visualized at: {url}")


In [ ]:
from os import path
import cognee


dataset_name = "explanations"
await cognee.add([path.abspath("../.test_data/Natural language processing.txt")], dataset_name)


dataset_name = "short_stories"
# data_directory_path is defined above
await cognee.add("data://" + data_directory_path, dataset_name)

text_1 = """A quantum computer is a computer that takes advantage of quantum mechanical phenomena.
At small scales, physical matter exhibits properties of both particles and waves, and quantum computing leverages this behavior, specifically quantum superposition and entanglement, using specialized hardware that supports the preparation and manipulation of quantum states.
Classical physics cannot explain the operation of these quantum devices, and a scalable quantum computer could perform some calculations exponentially faster (with respect to input size scaling) than any modern "classical" computer. In particular, a large-scale quantum computer could break widely used encryption schemes and aid physicists in performing physical simulations; however, the current state of the technology is largely experimental and impractical, with several obstacles to useful applications. Moreover, scalable quantum computers do not hold promise for many practical tasks, and for many important tasks quantum speedups are proven impossible.
The basic unit of information in quantum computing is the qubit, similar to the bit in traditional digital electronics. Unlike a classical bit, a qubit can exist in a superposition of its two "basis" states. When measuring a qubit, the result is a probabilistic output of a classical bit, therefore making quantum computers nondeterministic in general. If a quantum computer manipulates the qubit in a particular way, wave interference effects can amplify the desired measurement results. The design of quantum algorithms involves creating procedures that allow a quantum computer to perform calculations efficiently and quickly.
Physically engineering high-quality qubits has proven challenging. If a physical qubit is not sufficiently isolated from its environment, it suffers from quantum decoherence, introducing noise into calculations. Paradoxically, perfectly isolating qubits is also undesirable because quantum computations typically need to initialize qubits, perform controlled qubit interactions, and measure the resulting quantum states. Each of those operations introduces errors and suffers from noise, and such inaccuracies accumulate.
In principle, a non-quantum (classical) computer can solve the same computational problems as a quantum computer, given enough time. Quantum advantage comes in the form of time complexity rather than computability, and quantum complexity theory shows that some quantum algorithms for carefully selected tasks require exponentially fewer computational steps than the best known non-quantum algorithms. Such tasks can in theory be solved on a large-scale quantum computer whereas classical computers would not finish computations in any reasonable amount of time. However, quantum speedup is not universal or even typical across computational tasks, since basic tasks such as sorting are proven to not allow any asymptotic quantum speedup. Claims of quantum supremacy have drawn significant attention to the discipline, but are demonstrated on contrived tasks, while near-term practical use cases remain limited.
"""

text_2 = """A large language model (LLM) is a language model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification. LLMs acquire these abilities by learning statistical relationships from text documents during a computationally intensive self-supervised and semi-supervised training process. LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word.
LLMs are artificial neural networks. The largest and most capable, as of March 2024, are built with a decoder-only transformer-based architecture while some recent implementations are based on other architectures, such as recurrent neural network variants and Mamba (a state space model).
Up to 2020, fine tuning was the only way a model could be adapted to be able to accomplish specific tasks. Larger sized models, such as GPT-3, however, can be prompt-engineered to achieve similar results.[6] They are thought to acquire knowledge about syntax, semantics and "ontology" inherent in human language corpora, but also inaccuracies and biases present in the corpora.
Some notable LLMs are OpenAI's GPT series of models (e.g., GPT-3.5 and GPT-4, used in ChatGPT and Microsoft Copilot), Google's PaLM and Gemini (the latter of which is currently used in the chatbot of the same name), xAI's Grok, Meta's LLaMA family of open-source models, Anthropic's Claude models, Mistral AI's open source models, and Databricks' open source DBRX.
"""

dataset_name = "explanations"
await cognee.add(
    [
        text_1,
        text_2
    ],
    dataset_name
)

graph = await cognee.cognify(dataset_name)
